# Tuturial 1: Creating and Solving Factor Graphs
In this tutorial you will lear the basics of factor graphs and consepts such as variables, factors, priors and how create solve and explore them.

## What is a Factor Graph

Wikipedia tels us a factor graph is a bipartite graph representing the factorization of a function.
 
If you haven't taken some advanced statistics classes you are probably wondering, bipartite graph?, factorization?, I just want to navigate my robot.
For robotics, factor graphs are the common language used to describe your robot's estimation problem (the function) in a way both humans and computers can understand. 
The estimation problem can be anything from robot localization, structure from motion, calibration to full PoseSLAM with parametric or non-parametric measurements and beliefs.
A factor graph is a graphical model with two types of nodes, variables and factors, connected by edges between the variables and factors (bipartite graph). 
Variables represent the unknown random variables in the estimation problem, such as vehicle or landmark positions, sensor calibration parameters, and more.
Factors represent the algebraic interaction between particular variables, such as wheel odometry, which is captured through edges. 
Breaking down the complex system describing your robot to a graph of variables and factors is the factorization of the function decribing your system, such as the position and orientation (pose) of your robot at any given time. 
This factorisation allows us to solve the optimization problem for every variable given every measurement described by the factors.
See Caesar.jl docs [Graph Concepts](https://juliarobotics.org/Caesar.jl/latest/concepts/concepts/#Graph-Concepts) for more detail.

### Lets build a factor graph

First we will need some packages: 
- `NavAbilitySDK` - for interacting with the graph.
- `numpy` - for matrices and calculations.
- `Plotly` - for visualizing graphs [TODO].


In [ ]:
from navability.entities import *
from navability.services import *
from uuid import uuid4
import asyncio
import numpy as np

To create a new factor graph with default settings create a new NavAbility client and a client that indicates the user, robot, and session. 

In [ ]:
# Start with an empty factor graph
navability_client = NavAbilityHttpsClient()
client = Client("Guest", "SDKpy_" + str(uuid4())[0:4], "Tutorial1_" + str(uuid4())[0:4])
print(client)

## Variable and Factors 

Variables, denoted as the larger nodes in the figure below, represent state variables of interest such as vehicle or landmark positions, sensor calibration parameters, and more. Variables are likely hidden values which are not directly observed, but we want to estimate them from observed data and at least some minimal algebra structure from probabilistic measurement models.

Factors, the smaller nodes in the figure below, represent the algebraic interaction between particular variables, which is captured through edges. Factors must adhere to the limits of probabilistic models – for example conditional likelihoods capture the likelihood correlations between variables; while priors (unary to one variable) represent absolute information to be introduced. 

We start with a `Pose2` variable, i.e. position and orientation in two dimensions.
To add variables to our factor graph e created above, call `addVariable` with a label `x0` and type `Pose2`


In [ ]:
v = Variable("x0", VariableType.Pose2.value)
result_id = await addVariable(navability_client, client, v)
print(f"Added {v.label} with result ID {result_id}")
# Wait for it to be loaded.
await waitForCompletion(navability_client, [result_id])

We now have a factor graph with one variable, but to solve it we need some additional information. 
In this example we need the estimated starting point of our robot.
We use unary factors called priors to represent absolute information to be introduced.
This unary factor is taken as a matrix `FullNormal` distribution (a.k.a. a [multivariate normal distribution](https://en.wikipedia.org/wiki/Multivariate_normal_distribution)).  Let's create a `PriorPose2` unary factor with zero mean and a covariance matrix of (`diagm([0.05,0.05,0.01].^2)`):

$\mu = \begin{bmatrix} 0 & 0 & 0 \end{bmatrix}^T$

$\Sigma = \begin{bmatrix} 0.0025 & 0.0 &  0.0 \\ 0.0 & 0.0025 & 0.0 \\ 0.0 &  0.0 &  0.0001 \end{bmatrix}$

In [ ]:
prior_distribution = FullNormal(mu=np.zeros(3), cov=np.power(np.diag([0.1, 0.1, 0.1]),2))
prior_factor = Factor("x0f1", "PriorPose2", ["x0"], FactorData(fnc=PriorPose2(Z=prior_distribution).dump()))
result_id = await addFactor(navability_client, client, prior_factor) 
print(f"Added {v.label} with result ID {result_id}")
# Wait for it to be loaded.
await waitForCompletion(navability_client, [result_id])

We can look at the factor graph we have so far using the generated link to the NavAbility WebApp

In [ ]:
print("https://app.navability.io/cloud/graph/?userId=%s&robotStartsWith=%s&sessionStartsWith=%s" % (client.userId, client.robotId, client.sessionId))

The prior is now connected to the variable, `x0`, but it is not initialized yet. Automatic initialization of variables depends on how the factor graph model is constructed. So far, `x0` has not been initialized.

**Graph-based Initialization**

Why is `:x0` not initialized? Since no other variable nodes have been 'connected to' (or depend) on `:x0` and future intentions of the user are unknown, the initialization of `:x0` is deferred until the latest possible moment. `IncrementalInference.jl` assumes that the user will generally populate new variable nodes with most of the associated factors before moving to the next variable. 

By delaying initialization of a new variable (say `:x0`) until a second newer uninitialized variable (say `:x1`) depends on `:x0`, the IncrementalInference algorithms hope to then initialize `:x0` with the more information from previous and surrounding variables and factors. Also note that initialization of variables is a local operation based only on the neighboring nodes – global inference over the entire graph is shows later in this tutorial.

**Robot Odometry - Relative Factor**

Next, we want to add an odometry factor that connects our two robot poses `x0` and `x1` together to form a chain.
Here we use a relative factor of type `Pose2Pose2` with a measurement from pose `x0` to `x1` of (x=1.0,y=0.0,θ=pi/2); the robot drove 1 unit forward (in the x direction).
Similarly to the prior we added above, we use an `FullNormal` distribution to represent the odometry with mean and covariance:

$\mu =(x=1, y=0, \theta=\frac{\pi}{2})$

$\Sigma = \begin{bmatrix} 0.01 & 0.0 &  0.0 \\ 0.0 & 0.01 & 0.0 \\ 0.0 &  0.0 &  0.0001 \end{bmatrix}$

In [ ]:
v = Variable("x1", VariableType.Pose2.value)
result_v = await addVariable(navability_client, client, v)
print(f"Added {v.label} with result ID {result_v}")
odo_distribution = FullNormal(mu=[1.0, 0.0, np.pi/2], cov=np.power(np.diag([0.1, 0.1, 0.01]),2))
fac_1 = Factor("x0x1f1", "Pose2Pose2", ["x0", "x1"], FactorData(fnc=Pose2Pose2(Z=odo_distribution).dump()))
result_fac_1 = await addFactor(navability_client, client, fac_1) 
print(f"Added {fac_1.label} with result ID {result_fac_1}")

# Wait for it to be loaded.
await waitForCompletion(navability_client, [result_v, result_fac_1])

Lets look at the factor graph again

In [ ]:
print("https://app.navability.io/cloud/graph/?userId=%s&robotStartsWith=%s&sessionStartsWith=%s" % (client.userId, client.robotId, client.sessionId))

### Solving

We now have a graph we can solve using the Multi-Modal iSAM (MM-iSAM) algorithm. 
The default solver will perform non-parametric inference/state-estimation over our newly created graph.

In [ ]:
solve_request = await solveSession(navability_client, client)
# Wait for it to be loaded.
await waitForCompletion(navability_client, [solve_request])

### Results

The NavAbility WebApp allows visualization of the belief state over any of the variables.  Also try with Belief and Distribution buttons to see more of the underlying posterior marginal belief estimates.

In [ ]:
print("https://app.navability.io/cloud/map/?userId=%s&robotStartsWith=%s&sessionStartsWith=%s" % (client.userId, client.robotId, client.sessionId))

**What is happening**

The figure shows the position and orientation (red forward) for poses `x0` and `x1`. As well as the covarance ellipse. 
A few other functoins are also handy for interacting with the factor graph, for instance `getVariable` returns the full variable.
Or if you are interested in the suggested Parametric Point Estimate (PPE) you can get this from the PPE data.

In [ ]:
variables = await ls(navability_client, client)
print(f"Variables in graph: {variables}")
ppes = {
        v: (await getVariable(navability_client, client, v)).ppes["default"].suggested
        for v in variables
    }
print(ppes)

**Parametric point estimates and beliefs**

A PPE can be the maximum or mean of the belief. 
If the belief is a normal distribution, both correspond with its mean. 
However, care should be taken with using PPEs when beliefs might be non-parametric, for example, in a multimodal belief with two peaks, max corresponds with the maximum of the two peaks while the mean will fall somewhere in between them. 
In non-parametric cases, it is better to work with the full belief obtained by the Kernel Density Estimate (KDE).  
Kernel Density Estimation is a non-parametric way to estimate the probability density function of a random variable.
With the default solver, a full probability density function is always available and can be visualized as shown by the distribution plot feature in the NavAbility WebApp.
Non-parametric solutions will be discussed in more detail in tutorial 2.

In [ ]:
print("https://app.navability.io/cloud/map/?userId=%s&robotStartsWith=%s&sessionStartsWith=%s" % (client.userId, client.robotId, client.sessionId))

More plotting options exist, depending on how you are accessing the data.  See the [section on Plotting in the Caesar docs](https://juliarobotics.org/Caesar.jl/latest/concepts/2d_plotting/) for additional detail.

### Adding more poses and a point landmark

So far we worked with the `Pose2` factor type. 
Among others, `NavAbilitySDK` also provides the `Point2` variable and `Pose2Point2BearingRange` factor types, which we will use to represent a landmark sighting in our factor graph.
We will add a landmark `l1` with bearing range measuremet of bearing=$(\mu=0,\sigma=0.03)$ range=$(\mu=0.5,\sigma=0.1)$ and continue our robot trajectory by driving around in a square.

In [ ]:
variables = [
  # Variable("l1", VariableType.Point2.value),
  Variable("x2", VariableType.Pose2.value),
  Variable("x3", VariableType.Pose2.value),
  Variable("x4", VariableType.Pose2.value)]
results_variables = [await addVariable(navability_client, client, v) for v in variables]

factors = [
  # TODO
  Factor("x0l1f2", "Pose2Point2BearingRange", ["x0", "x1"], 
      FactorData(fnc=Pose2Point2BearingRange(Normal(0.0,0.03), Normal(0.5,0.1)).dump())),
  Factor("x1x2f1", "Pose2Pose2", ["x1", "x2"], 
      FactorData(fnc=Pose2Pose2(odo_distribution).dump())),
  Factor("x2x3f1", "Pose2Pose2", ["x2", "x3"], 
      FactorData(fnc=Pose2Pose2(odo_distribution).dump())),
  Factor("x3x43f1", "Pose2Pose2", ["x3", "x4"], 
      FactorData(fnc=Pose2Pose2(odo_distribution).dump())),
]
results_factors = [await addFactor(navability_client, client, f) for f in factors]
await waitForCompletion(navability_client, results_variables + results_factors)

Lets go look at the factor graph now, again using the WebApp

In [ ]:
print("https://app.navability.io/cloud/graph/?userId=%s&robotStartsWith=%s&sessionStartsWith=%s" % (client.userId, client.robotId, client.sessionId))

We now have a longer odometry chain with one landmark sighting, let's solve the factor graph again so we can have a look at the results.

In [ ]:
solve_request = await solveSession(navability_client, client)
# Wait for it to be loaded.
await waitForCompletion(navability_client, [solve_request], maxSeconds=120)

The solve will take a bit of time.  Just keep watching the geometric visualization, which will automatically update as more of the solution is published

In [ ]:
print("https://app.navability.io/cloud/map/?userId=%s&robotStartsWith=%s&sessionStartsWith=%s" % (client.userId, client.robotId, client.sessionId))

Once solving has completed, the variable results can be looked at in various ways.  Here we are just looking at the PPE values, but the full posterior marginal beliefs are available by simply asking for more results.

In [ ]:
#pl = plotSLAM2D(fg, drawContour=false, drawEllipse=true, drawhist=false, drawPoints=false)
# TODO: Need l1 to solve for this to work correctly.
variables = await ls(navability_client, client)
print(f"Variables in graph: {variables}")
ppes = {
        v: (await getVariable(navability_client, client, v)).ppes.get("default") #.suggested
        for v in variables
    }
print(ppes)

### Adding a Loop Closure

As expected, the robot continued its square trajectory to end off where it started. 
To illustrate a loop closure, we add another bearing range sighting to from pose `x4` to landmark `l1`, solve the graph and plot the new results: 

In [ ]:
p2br = Factor("x4l1f1", "Pose2Point2BearingRange", ["x4", "l1"], 
          FactorData(fnc=Pose2Point2BearingRange(Normal(0.0,0.03), Normal(0.5,0.1)).dump()))
result_factor = await addFactor(navability_client, client, p2br)
await waitForCompletion(navability_client, [result_factor])

In [ ]:
solve_request = await solveSession(navability_client, client)
# Wait for it to be loaded.
await waitForCompletion(navability_client, [solve_request], maxSeconds=120)

Lets go look at the final results again, which now includes the loop closure.  Use the 'Show Belief', 'Show Distribution' buttons along the bottom for more visual information.  You can also trigger new solves (as long as the Global Filter fields are properly set in the far right filter menu).

Use the WebApp hamburger menu on the left to navigate between the graph and geometric map visualization.  You can also use the Global Filter menu on the right to set which information to visualize.

In [ ]:
print("https://app.navability.io/cloud/graph/?userId=%s&robotStartsWith=%s&sessionStartsWith=%s" % (client.userId, client.robotId, client.sessionId))

In [ ]:
print("https://app.navability.io/cloud/map/?userId=%s&robotStartsWith=%s&sessionStartsWith=%s" % (client.userId, client.robotId, client.sessionId))

## Next Steps

Tutorial 2 will give an introduction to non-parametric solutions.

## Additional Resources

### Variable representation and factor algebraic operations

The variables used in this tutiorail, `Point2` and `Pose2`, are represented as points on manifolds and all algebraic operations, inference/optimization are also performed on manifold.
For more information on how manifolds are used in Ceaser.jl, refer to the [manifold section in the documentation](https://juliarobotics.org/Caesar.jl/latest/concepts/using_manifolds/).

### Custom Variables and Factors

In most scenarios, the existing variables and factors should be sufficient for most robotics applications. 
Caesar however, is extensible and allows you to easily incorporate your own variable and factor types for specialized applications.

Have a look at the Caesar documentaton if you are interested in creating custom vairables, factors (or priors)

- [Custom variables](https://juliarobotics.org/Caesar.jl/latest/examples/custom_variables/)
- [Custom prior factors](https://juliarobotics.org/Caesar.jl/latest/examples/basic_definingfactors/)
- [Custom relative factors](https://juliarobotics.org/Caesar.jl/latest/examples/custom_relative_factors/)


### Bayes (Junction) Tree

Inference is performed on the bayes tree see: [Bayes tree principles](https://juliarobotics.org/Caesar.jl/latest/principles/bayestreePrinciples/)